In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10

In [3]:
from kaggle_secrets import UserSecretsClient
import os
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_Token")

os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
login(token=hf_token)

In [4]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [5]:
import sys
print(sys.executable)

/usr/bin/python3


In [6]:
import torch
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

12.4
90100
True
Tesla P100-PCIE-16GB


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForLanguageModeling,DataCollatorForSeq2Seq 
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

2025-06-09 17:30:35.228077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749490235.414057      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749490235.471263      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
ds = load_dataset("ShenLab/MentalChat16K")

print(ds['train'].column_names)

model_name = "google/flan-t5-large"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
    )

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v", "o", "wi", "wo"],
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained(model_name)

README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

['instruction', 'input', 'output']


...

trainable params: 10,027,008 || all params: 793,177,088 || trainable%: 1.2642


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def t5_preprocess(batch_examples):
    prompts = batch_examples['input']
    responses = batch_examples['output']

    input_texts = []
    target_texts = []
    
    for i in range(len(prompts)):
        user_content = str(prompts[i]) if prompts[i] is not None else ""
        assistant_content = str(responses[i]) if responses[i] is not None else ""

        input_text = f"Respond to this mental health query: {user_content}"
        target_text = assistant_content
        
        input_texts.append(input_text)
        target_texts.append(target_text)

    model_inputs = tokenizer(
        input_texts,
        truncation=True,
        max_length=256,  
        padding="max_length"
    )
    

    labels = tokenizer(
        target_texts,
        truncation=True,
        max_length=256,  
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

tokenized_dataset = ds.map(t5_preprocess, batched=True)

Map:   0%|          | 0/16084 [00:00<?, ? examples/s]

In [10]:
hyperparameters = TrainingArguments(
    output_dir="./T5-qlora-finetuned-results", 
    num_train_epochs=3,
    per_device_train_batch_size=8,      
    gradient_accumulation_steps=2,    
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    warmup_ratio=0.03,  
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir="./logs-qlora",          
    logging_steps=10,
    logging_strategy="steps",
    save_strategy="epoch",
    bf16=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    disable_tqdm=False,           
    report_to=[],                    
    dataloader_pin_memory=False
)


In [11]:
trainer = Trainer(
    model=model,
    args=hyperparameters,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model)
)

trainer.train()

/tmp/ipykernel_19/2175170608.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,4.953700
20,4.823500
30,4.226100
40,5.030300
50,4.316600
60,4.627700
70,4.909200
80,5.113200
90,4.568900
100,3.570000


TrainOutput(global_step=3015, training_loss=1.71620836463536, metrics={'train_runtime': 18577.4581, 'train_samples_per_second': 2.597, 'train_steps_per_second': 0.162, 'total_flos': 5.630594372547379e+16, 'train_loss': 1.71620836463536, 'epoch': 2.9975136747886624})

In [12]:
trainer.save_model("/kaggle/working/fine_tuned_model")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_model")
print(os.listdir("/kaggle/working/"))
print(os.listdir("/kaggle/working/fine_tuned_model"))

['fine_tuned_model', '__notebook__.ipynb', 'T5-qlora-finetuned-results']
['special_tokens_map.json', 'adapter_model.safetensors', 'spiece.model', 'adapter_config.json', 'training_args.bin', 'README.md', 'tokenizer_config.json', 'tokenizer.json']


In [13]:
# from huggingface_hub import HfApi, create_repo

# api = HfApi()
# repo_name = "syurmen/T5-finetuned"

# api.upload_folder(
#     folder_path="/kaggle/working/fine_tuned_model",
#     repo_id=repo_name,
#     repo_type="model"
# )